In [26]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [27]:
data = pd.read_excel('data_ford_price.xlsx') 

In [28]:
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

y = data['price']
x = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

Создадим модель и обучим модель ее как есть, до того как выделять приоритетные признаки

In [29]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4682.957


Выделим 3 приоритетных признака по методу RFE

In [30]:
from sklearn.feature_selection import RFE

In [31]:
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'lat'], dtype=object)

Теперь обучим модель на выбранных признаках и посмотрим какая получилась метрика

In [32]:
X_train = X_train[['year','cylinders','lat']]
X_test = X_test[['year','cylinders','lat']]

In [33]:
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 5096.570


Метрика по сравнению с исходной ухудшилась. Мы потеряли часть информации при отборе только 3х признаков

Реализация SelectKBest

In [34]:
from sklearn.feature_selection import SelectKBest, f_regression

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [35]:
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'odometer'], dtype=object)

Теперь обучим модель на выбранных признаках и посмотрим какая получилась метрика

In [36]:
X_train = X_train[['year','cylinders','odometer']]
X_test = X_test[['year','cylinders','odometer']]

In [37]:
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4708.946


Мы потеряли информацию, но значительно меньше, чем по методу RFE

Выводы (сравнение результатов)

1. Оба метода отбора признаков потеряли часть информации и ухудшили значение метрики от исходного. Это говорит о том, что в данной задаче больше важных признаков, чем 3.

2. Метод SelectKBest сработал в данном случае существенно лучше, значение метрики MSE на тестовой выборке лучше и существенно